[Back to Index](index.ipynb)

## Generate Summary Statistics for Sacramento County

Begin by importing all python modules we will need.

In [ ]:
# A 'magic' command to display plots inline
%matplotlib inline

import requests
import pandas as pd
from datetime import datetime
import itertools
import calendar
import seaborn as sns

### 1. Get projections

The code in the next cell contains a bunch of functions to:
    - fetch annual averages of Max. Temp, Min. Temp and Precipitation for 4 GCMs and 2 scenarios for Sacramento County from the API
    - convert units
    - return a new dataframe that contains all the data
    
Just run the cell, you can go over the code at your leisure.

In [ ]:
def kelvin_to_F(val):
    return  (val - 273.15) * 9/5 + 32

def kgm2s_to_inchyear(row):
    year = row.event.year
    days = 366 if calendar.isleap(year) else 365
    return (row.image * 86400) * 0.0393701 * days

def get_projections():
    # Create an empty list to hold dataframes
    df_list = []

    # Make a combined list of GCMs, scenarios, climate variables for looping
    climvar = ['tasmax', 'tasmin', 'pr']
    period = ['year']
    models = ['CanESM2', 'CNRM-CM5', 'HadGEM2-ES', 'MIROC5']
    scenarios = ['rcp45', 'rcp85']
    zipped = itertools.product(climvar, period, models, scenarios)

    # Request parameters
    params = {'pagesize': 94, 'stat': 'mean', 'ref': '/api/counties/34/'}
    
    # Loop through zipped
    for item in zipped:
        # Create slug
        slug = '_'.join(item)
        url = 'http://api.cal-adapt.org/api/series/%s/rasters/' % slug
        # Make request
        response = requests.get(url, params=params)

        # Get data
        if response.ok:
            print('Processing:', slug)
            data = response.json()
            # Create temp dataframe
            df = pd.DataFrame(data['results'])
            # Change format of `event` field to datetime
            df['event'] = pd.to_datetime(df['event'], format='%Y-%m-%d')
            # The data from API should be sorted, but sort ascending to be sure
            df = df.sort_values('event')  
            # Convert units
            if 'tas' in slug:
                df.image = df.image.apply(lambda x: kelvin_to_F(x))
            else:
                df.image = df.apply(kgm2s_to_inchyear, axis=1)
            # Discard all columns except event and image
            df = df[['event', 'image']]
            df['climvar'] = item[0]
            df['model'] = item[2]
            df['scenario'] = item[3]
            df_list.append(df)          
        else:
            print('Failed:', slug)
    # Combine all the dataframes into one and return
    return pd.concat(df_list)

Execute the `get_projections()` function. If all goes well, you should see a list of messages as the code process each timeseries.

In [ ]:
projections = get_projections()

Explore the `projections` dataframe

In [ ]:
projections = projections.reset_index(drop=True)
projections.tail()

### 2. Get historical observed data

The code in the next cell contains a bunch of functions to:
    - fetch annual averages of Max. Temp, Min. Temp and Precipitation for the livneh data for Sacramento County from the API
    - convert units
    - return a new dataframe that contains all the data
    
Just run the cell, you can go over the code at your leisure.

In [ ]:
def celsius_to_F(val):
    return val * 9/5 + 32 

def mmday_to_inchyear(row):
    year = row.event.year
    days = 366 if calendar.isleap(year) else 365
    return row.image * 0.0393701 * days

def get_observed():
    # Create an empty list to hold dataframes
    df_list = []

    # Make a combined list of GCMs, scenarios, climate variables for looping
    climvar = ['tasmax', 'tasmin', 'pr']
    period = ['year']
    zipped = itertools.product(climvar, period, ['livneh'])
    
    # Request parameters
    params = {'pagesize': 64, 'stat': 'mean', 'ref': '/api/counties/34/'}

    # Loop through zipped
    for item in zipped:
        # Create slug
        slug = '_'.join(item)
        url = 'http://api.cal-adapt.org/api/series/%s/rasters/' % slug 
        # Make request
        response = requests.get(url, params=params)

        # Get data
        if response.ok:
            print('Processing:', slug)
            data = response.json()
            # Create temp dataframe
            df = pd.DataFrame(data['results'])
            # Change format of `event` field to datetime
            df.event = pd.to_datetime(df.event, format='%Y-%m-%d')
            # The data from API should be sorted, but sort ascending to be sure
            df = df.sort_values('event')     
            # Convert units
            if 'tas' in slug:
                df.image = df.image.apply(lambda x: celsius_to_F(x))
            else:
                df.image = df.apply(mmday_to_inchyear, axis=1)
            col_list = ['event', 'image']
            df = df[col_list]
            df['climvar'] = item[0]
            df['model'] = 'livneh'
            df['scenario'] = 'historical'
            df_list.append(df)          
        else:
            print('Failed:', slug)
    # Combine all the dataframes into one and return
    return pd.concat(df_list)

Execute the `get_observed()` function. If all goes well, you should see a list of messages as the code process each timeseries

In [ ]:
observed = get_observed()
observed = observed.reset_index(drop=True)

Explore the observed dataframe

In [ ]:
observed.tail()

### 3. Summary stats

Combine projections and observed data

In [ ]:
df = pd.concat([projections, observed])
df = df.reset_index(drop=True)

In [ ]:
df.tail()

Generate summary stats for baseline period 1961-1990

In [ ]:
baseline = df[(df.event >= '1961-01-01') & (df.event <= '1990-01-01')]
baseline = baseline.groupby(['climvar', 'scenario', 'model'])['image'].agg(['mean', 'max', 'min', 'std'])
# groupby return a groupy object, convert it to a dataframe
df_baseline = pd.DataFrame(baseline)
df_baseline

Generate summary stats for 2020-2050

In [ ]:
projections_2020_2050 = df[(df.event >= '2020-01-01') & (df.event <= '2050-01-01')]
projections_2020_2050 = projections_2020_2050.groupby(['climvar', 'scenario', 'model'])['image'].agg(['mean', 'max', 'min', 'std'])
# Convert groupby object to dataframe
df_2020_2050 = pd.DataFrame(projections_2020_2050)
df_2020_2050

In [ ]:
quantiles_2020_2050 = df[(df.event >= '2020-01-01') & (df.event <= '2050-01-01')]
quantiles_2020_2050 = quantiles_2020_2050.groupby(['climvar', 'scenario', 'model'])['image'].quantile([0.1, 0.5, 0.9])
# Convert groupby object to dataframe
q_2020_2050 = pd.DataFrame(quantiles_2020_2050)
q_2020_2050

In [ ]:
projections_2070_2099 = df[(df.event >= '2070-01-01') & (df.event <= '2099-01-01')]
projections_2070_2099 = projections_2070_2099.groupby(['climvar', 'scenario', 'model'])['image'].agg(['mean', 'max', 'min', 'std'])
# Convert groupby object to dataframe
df_2070_2099 = pd.DataFrame(projections_2070_2099)
df_2070_2099

In [ ]:
quantiles_2070_2099 = df[(df.event >= '2070-01-01') & (df.event <= '2099-01-01')]
quantiles_2070_2099 = quantiles_2070_2099.groupby(['climvar', 'scenario', 'model'])['image'].quantile([0.1, 0.5, 0.9])
# Convert groupby object to dataframe
q_2070_2099 = pd.DataFrame(quantiles_2070_2099)
q_2070_2099

### 4. Export to Excel

In [ ]:
writer = pd.ExcelWriter('output.xlsx')
df_baseline.to_excel(writer,'Baseline')
df_2020_2050.to_excel(writer,'2020-2050')
q_2020_2050.to_excel(writer,'2020-2050 Quantiles')
df_2070_2099.to_excel(writer,'2070-2099')
q_2070_2099.to_excel(writer,'2070-2099 Quantiles')
writer.save()

### 5. Make charts

Create a linechart showing all timeseries for Max. Temperature and RCP 4.5. Export it to a png file.

In [ ]:
tasmax_rcp45 = df.loc[(df['climvar'] == 'tasmax') & (df['scenario'] != 'rcp85')]
tasmax_rcp45 = tasmax_rcp45.pivot(index='event', columns='model', values='image')
plot = tasmax_rcp45.plot()
fig = plot.get_figure()
fig.savefig('output.png')

Categorical [Scatterplots](https://seaborn.pydata.org/tutorial/categorical.html#categorical-scatterplots)

In [ ]:
tasmax_rcp85 = df.loc[(df['climvar'] == 'tasmax') & (df['scenario'] != 'rcp45')]
sns.stripplot(x="model", y="image", data=tasmax_rcp85, jitter=True)

In [ ]:
tasmax = df.loc[df['climvar'] == 'tasmax']
sns.swarmplot(y="model", x="image", hue="scenario", data=tasmax)

In [ ]:
precip = df.loc[df['climvar'] == 'pr']
sns.swarmplot(y="model", x="image", hue="scenario", data=precip)

In [ ]:
sns.boxplot(y="model", x="image", hue="scenario", data=tasmax)

In [ ]:
tas = df.loc[(df['climvar'] == 'tasmax') | (df['climvar'] == 'tasmin')]
sns.factorplot(y="model", x="image", hue="scenario", kind="box", col="climvar", data=tas)